In [2]:
import pandas as pd
import os
import glob

# Function to process single BI Rate CSV file
def process_bi_rate_file(file_path):
    # Read CSV file, skip first 2 rows (Variabel and BI Rate headers)
    df = pd.read_csv(file_path, skiprows=3, encoding='utf-8')
    
    # Extract year from filename
    year = file_path.split('/')[-1].split('.')[0].split(',')[1].strip()
    
    # Select only the BI Rate row and monthly values
    bi_rate_row = df.iloc[0, 1:13].values  # Skip the 'BI Rate' column name and 'Tahunan'
    
    # Create Series with proper index
    months = ['Januari', 'Februari', 'Maret', 'April', 'Mei', 'Juni', 
             'Juli', 'Agustus', 'September', 'Oktober', 'November', 'Desember']
    series = pd.Series(bi_rate_row, index=pd.to_datetime([f'{year}-{i+1}-01' for i in range(12)]))
    
    return series

# Directory containing the BI Rate CSV files
data_dir = '/Users/hakam/Documents/Hakam Raves M.Kom M.E/Thesis/macrodataset'  # Update this to your data directory path

# Get all BI Rate CSV files
file_pattern = 'BI Rate, *.csv'
files = sorted(glob.glob(os.path.join(data_dir, file_pattern)))

# Process all files and combine data
all_data = []
for file in files:
    try:
        series = process_bi_rate_file(file)
        all_data.append(series)
    except Exception as e:
        print(f"Error processing {file}: {e}")

# Combine all series into a single DataFrame
if all_data:
    combined_df = pd.concat(all_data)
    
    # Sort by date
    combined_df = combined_df.sort_index()
    
    # Create output DataFrame with desired format
    output_df = pd.DataFrame({
        'Date': combined_df.index.strftime('%Y-%m-%d'),
        'BI_Rate': combined_df.values
    })
    
    # Save to CSV
    output_df.to_csv('BI_Rate_Combined_2015_2025.csv', index=False)
    
    # Display first few rows
    display(output_df.head())
else:
    print("No data files were processed successfully.")

,Date,BI_Rate
0,2013-01-01,5.75
1,2013-02-01,5.75
2,2013-03-01,5.75
3,2013-04-01,5.75
4,2013-05-01,5.75


In [3]:
import pandas as pd
import os
import glob

def process_money_supply_file(file_path):
    # Read CSV file, skip first 2 rows (Money Type and Money Supply headers)
    df = pd.read_csv(file_path, skiprows=3, encoding='utf-8')
    
    # Extract year from filename
    year = file_path.split('/')[-1].split('.')[0].split(',')[1].strip()
    
    # Select M1 and M2 rows (Total M1 and Total M2)
    m1_row = df[df.iloc[:, 0].str.contains('Total \(M1', na=False)].iloc[0, 1:13].values
    m2_row = df[df.iloc[:, 0].str.contains('Total \(M2', na=False)].iloc[0, 1:13].values
    
    # Create date index
    dates = pd.to_datetime([f'{year}-{i+1}-01' for i in range(12)])
    
    # Create Series for both M1 and M2
    m1_series = pd.Series(m1_row, index=dates)
    m2_series = pd.Series(m2_row, index=dates)
    
    return m1_series, m2_series

# Directory containing the Money Supply CSV files
data_dir = '/Users/hakam/Documents/Hakam Raves M.Kom M.E/Thesis/macrodataset'

# Get all Money Supply CSV files
file_pattern = 'Money Supply, *.csv'
files = sorted(glob.glob(os.path.join(data_dir, file_pattern)))

# Process all files and combine data
all_m1_data = []
all_m2_data = []

for file in files:
    try:
        m1_series, m2_series = process_money_supply_file(file)
        all_m1_data.append(m1_series)
        all_m2_data.append(m2_series)
    except Exception as e:
        print(f"Error processing {file}: {e}")

# Combine all series into DataFrames
if all_m1_data and all_m2_data:
    # Combine and sort M1 data
    combined_m1 = pd.concat(all_m1_data).sort_index()
    combined_m2 = pd.concat(all_m2_data).sort_index()
    
    # Create output DataFrame with desired format
    output_df = pd.DataFrame({
        'Date': combined_m1.index.strftime('%Y-%m-%d'),
        'M1': combined_m1.values,
        'M2': combined_m2.values
    })
    
    # Save to CSV
    output_df.to_csv('Money_Supply_Combined_2013_2025.csv', index=False)
    
    # Display first few rows
    display(output_df.head())
else:
    print("No data files were processed successfully.")

,Date,M1,M2
0,2013-01-01,787859.68,3268789.15
1,2013-02-01,786548.67,3280420.25
2,2013-03-01,810054.88,3322528.96
3,2013-04-01,832213.49,3360928.07
4,2013-05-01,822876.47,3426304.92
